## This notebook loads ERA40 and WATCH focing data for GRAND dams inflow forecast
- ERA40: Volumetric Soil Water (SWVL)
- WATCH: Snowfall (Snowf)

Donghoon Lee @ May-23-2020

In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr
from netCDF4 import num2date, Dataset
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import rasterio
import gdal
import snetk
import datetime
from tools import DownloadFromURL

### Loading ERA40 Volumetric Soil Water (SWVL) Layer 1-4
The unit of SWVL is $m^3m^{-3}$. 

In [2]:
filn_swvl = '/Users/dlee/data/era/swvl_mon.nc'
if not os.path.exists(filn_swvl):
    # Load NetCDF files
    ds = xr.open_mfdataset('/Users/dlee/data/era/swvl/era_sm_*.nc')
    # Sums 4 layers
    ds_month = ds.swvl1 + ds.swvl2 + ds.swvl3 + ds.swvl4
    ds_month.name = 'swvl'
    # Selecting time period from 1958-01 to 2000-12
    swvl_mon = ds_month.loc['1958-01':'2000-12']
    if False:
        # Moving average (seasonal mean)
        ds_season = ds_month.rolling(time=3, center=True).mean().dropna('time')
        ds_season.name = 'swvl'
        swvl_mo3 = ds_season.loc['1958-01':'2000-12']
    # Save to NetCDF file
    swvl_mon.to_netcdf(filn_swvl)
    print('%s is saved.' % filn_swvl)

### Loading WATer and global CHange (WATCH) Forcing Data (WFD) - 20th Century data: Snowfall (Snowf)
The original (time resoultion is second) unit of Snowf is $kg m^{-2}s^{-1}$. This is aggregated at monthly scale, so the final unit is $kgm^{-2}$.

In [3]:
filn_snow = '/Users/dlee/data/watch/snowf_mon.nc'
if not os.path.exists(filn_snow):
    # Load NetCDF files
    ds = xr.open_mfdataset('/Users/dlee/data/watch/snowf/Snowf_WFD_GPCC_*.nc', decode_times=False, decode_cf=True)
    # Fix the DatetimeIndex of the Xarray Dataset
    time_fixed = pd.date_range('1958-01-01 00:00:00', periods=ds.time.shape[0], freq='3H')
    ds = ds.drop(['time','timestp','nav_lon','nav_lat'])
    ds = ds.assign_coords(time = time_fixed)
    ds = ds.rename_dims({'land':'land', 'tstep':'time'})
    # Resample
    snow_mon = ds.resample(time="1M").sum()
    # Save to NetCDF file
    snow_mon.to_netcdf(filn_snow)
    print('%s is saved.' % filn_snow)

### Reshaping
- ERA40 longitude starts from 0 degree, so we shift the western half to the east.
- Reshape to global 2D data (516 tims, 259200 grids)

In [4]:
## Reshaping ERA40-swvl data
# Loading NetCDF file
nc = Dataset(filn_swvl, 'r')
swvl = np.array(nc.variables['swvl'])
lat = np.array(nc.variables['latitude'])
lon = np.array(nc.variables['longitude'])
tim = nc.variables['time']
tim = num2date(tim[:], tim.units, tim.calendar)
# Shift the western half to the east
swvl = np.dstack((swvl[:,:,360::], swvl[:,:,0:360]))
swvl = swvl[:,:-1,:]
lon = np.hstack((lon[360::] - 360, lon[0:360]))
swvlCopy = swvl.copy()
# Make ERA40-swvl land-mask
lsm_swvl = ~(swvl[1,:,:] == np.min(swvl[1,:,:])) + 0
# Reshape to 2D format
swvl = swvl.reshape([swvl.shape[0], swvl.shape[1]*swvl.shape[2]])

## Reshaping WFD-snowf data
# Loading NetCDF file
nc = Dataset(filn_snow, 'r')
snow_raw = np.array(nc['Snowf'])
land = np.array(nc['land'])
# Reshape to 2D format
snow = np.zeros([516, 360*720])
snow[:,np.int32(land)-1] = snow_raw

### Remove data in ocean area
While we calculate summation and rolling averages in 'Xarray', the missing values have been changed.</br>
Here, we will find grids with approximately zero (probabily NaN) values, then set those grids as missing data (zero).</br>
There still are some missing values (e.g.,negative), but we ignore first.

In [5]:
## ERA40 SWVL
# Find approximately zero
rdx_swvl = (np.sum(np.abs(swvl) < 1e-05, axis=0) == 516)     # 97270 grids alive
# Replace those grids to zero
swvl[:,rdx_swvl] = 0

## WFD Snowf
rdx_snow = np.setdiff1d(np.arange(0,360*720), np.int32(land)-1)

### Basin-scale accumulated data using DDM30 upsteam network
Here, we accumulate the data using global streamflow network (DDM30) and flow accumulation algorithm.

In [6]:
# Load streamflow network (DDM30)
with rasterio.open('./data/DDM30_fdir_rec.tif') as src:
    fdir = src.read(1)
# Aggregate upstream swvl
filn_swvl_up = '/Users/dlee/data/era/swvl_mon_up_ddm30.npz'
filn_snow_up = '/Users/dlee/data/watch/snowf_mon_up_ddm30.npz'
if not (os.path.exists(filn_swvl_up) & os.path.exists(filn_snow_up)):
    # Create aggregated upstream soil moisture
    swvl_up = np.zeros(swvl.shape)
    snow_up = np.zeros(snow.shape)
    for i in range(swvl_up.shape[0]):
        # Convert table format to map format
        swvl_map = np.reshape(swvl[i,:], [360,720])
        snow_map = np.reshape(snow[i,:], [360,720])
        # Remove the latitudes where the flow direction does not exist
        # *12 and 292 are manually checked
        swvl_map = swvl_map[12:292,:] 
        snow_map = snow_map[12:292,:] 
        # Accumulate soil moisture with flow accumulation
        swvl_acc = snetk.flowAccumulate(fdir, swvl_map)
        snow_acc = snetk.flowAccumulate(fdir, snow_map)
        # Add the latitudes for being original size
        swvl_acc = np.vstack((np.zeros([12,720]), swvl_acc, np.zeros([68,720])))
        snow_acc = np.vstack((np.zeros([12,720]), snow_acc, np.zeros([68,720])))
        # Convert map format to table format
        swvl_up[i,:] = np.reshape(swvl_acc, [1,360*720])
        snow_up[i,:] = np.reshape(snow_acc, [1,360*720])
        print("%d/%d is processed.." % (i+1, swvl_up.shape[0]))
        
    # Remove the identify NaN grids
    swvl_up[:,rdx_swvl] = 0
    snow_up[:,rdx_snow] = 0
    # Save the accumulated upstream soil moisture
    np.savez_compressed(filn_swvl_up, swvl_up=swvl_up)
    print('%s is saved.' % filn_swvl_up)
    np.savez_compressed(filn_snow_up, snow_up=snow_up)
    print('%s is saved.' % filn_snow_up)

### Select data at GRAND Dams Grids
Here, we select data at the grids that 1,593 GranD Dams located.

In [7]:
# Load the accumulated upstream soil moisture
swvl_up = np.load(filn_swvl_up)['swvl_up']
snow_up = np.load(filn_snow_up)['snow_up']

# Load the selected 1,593 GranD Dams
ind_dams = np.load('./data/ind_dams.npz')['ind_dams']
damList = ind_dams[0,:]
ind_dams = ind_dams[1,:]

In [8]:
# Select swvl_up values at the 1,593 dam locations
swvl_up_dams = swvl_up[:,ind_dams]
snow_up_dams = snow_up[:,ind_dams]
# Check grids with all 0 (NaN) values (3 points are found)
no_swvl = (np.sum(swvl_up_dams < 1e-05, axis=0) > 100)

In [9]:
# Save as Pandas DataFrame
mondex = pd.period_range('{:04d}-{:02d}'.format(1958,1), periods=516, freq='M')
dfSwvlDams = pd.DataFrame(swvl_up_dams, index=mondex, columns=damList)
dfSnowDams = pd.DataFrame(snow_up_dams, index=mondex, columns=damList)
filn = './data/dfSwvlDams.hdf'
dfSwvlDams.to_hdf(filn, key='df', complib='blosc:zstd', complevel=9); print('%s is saved.' % filn)
filn = './data/dfSnowDams.hdf'
dfSnowDams.to_hdf(filn, key='df', complib='blosc:zstd', complevel=9); print('%s is saved.' % filn)

./data/dfSwvlDams.hdf is saved.
./data/dfSnowDams.hdf is saved.


### Land-mask comparison

In [ ]:
# # Compared to PCRGLOB-WB's global coverage, ERA40-swvl has missing values in 
# # some areas (e.g., Central Africa dessert, the Great lakes, etc.)
# #
# # Load ERA40 land-mask
# nc = Dataset(os.path.join('data', 'era', 'lsm.nc'), 'r')
# lsm_era = np.array(nc.variables['lsm']).astype(np.int16)
# lsm_era = np.squeeze(lsm_era); lsm_era = lsm_era[:-1,:]
# lsm_era = np.hstack((lsm_era[:,360::], lsm_era[:,0:360]))
# # Load PCRGLOB-WB land-mask
# lsm_pcr = np.load('./data/landMask_PCRGLOBWB.npy')      # From discharge data
# lsm_pcr[lsm_pcr != 1] = 0
# # Load WATCH land-mask
# nc = Dataset(os.path.join('data','watch','SnowCRs-land.nc'))
# ldc_wat = np.array(nc.variables['land'])
# lsm_wat = np.zeros([360,720])
# lsm_wat[np.unravel_index(ldc_wat-1, [360,720], order='C')] = 1
# # Generate map of dam locations (as grids)
# lsm_dams = np.zeros([360,720]).astype('int16')
# lsm_dams[np.unravel_index(ind_dams, [360,720])] = 1     # Total 1,261 grids    

# # Number of land grids
# print('=============================')
# print('Number of land grids')
# print('-----------------------------')
# print('ERA40:\t\t{:,}'.format(np.sum(lsm_era == 1)))        # 86,840
# print('ERA40-swvl:\t{:,}'.format(np.sum(lsm_swvl == 1)))    # 97,247
# print('WATCH:\t\t{:,}'.format(np.sum(lsm_wat == 1)))        # 67,420
# print('PCRGLOB-WB:\t{:,}'.format(np.sum(lsm_pcr == 1)))     # 66,273
# print('1,593 dams:\t{:,}'.format(np.sum(lsm_dams == 1)))    # 1,261
# print('=============================')
# # =============================================================================
# # # Create GeoTiff maps
# # import gdal
# # import snetk
# # lsm_pcr = lsm_pcr.astype('int16')
# # out_ds = snetk.make_raster360('lsm_pcr.tif', lsm_pcr, gdal.GDT_Int16, nodata=0); del out_ds
# # out_ds = snetk.make_raster360('lsm_era.tif', lsm_era, gdal.GDT_Int16, nodata=0); del out_ds
# # out_ds = snetk.make_raster360('lsm_swvl.tif', lsm_swvl, gdal.GDT_Int16, nodata=0); del out_ds
# # out_ds = snetk.make_raster360('lsm_dams.tif', lsm_dams, gdal.GDT_Int16, nodata=0); del out_ds
# # =============================================================================


# #%% Correlation between streamflow and soil moisture datasets
# # Two soil moisture datasets are considered:
# # - (a) grid specific soil moisture
# # - (b) accumulated upstream soil moisture
# from scipy.stats import pearsonr
# filnFlow = Path('/home/dlee/gdrive/gflood/data/flow.npy')
# flow = np.load(filnFlow).transpose()
# corrTable = np.zeros([flow.shape[1], 4])
# for i in range(flow.shape[1]):
#     corrTable[i,0:2] = pearsonr(flow[:,i], swvl[:,ind_pcr[i]])
#     corrTable[i,2:] = pearsonr(flow[:,i], swvl_up[:,ind_pcr[i]])
# # Print results
# print('-'*30)
# print('Number of significant grids')
# print('swvl:\t\t{:,}'.format(np.sum(corrTable[:,1] < 0.05)))
# print('swvl_up:\t{:,}'.format(np.sum(corrTable[:,3] < 0.05)))
# print('-'*30)

### Download WATer and global CHange (WATCH) Forcing Data (WFD) - 20th Century data

In [ ]:
# # WATCH Forcing data
# path_local = '/Users/dlee/data/watch/snowf/'
# path_watch = 'https://catalogue.ceh.ac.uk/datastore/eidchub/6b7a08f0-00f7-4c48-ac4d-e483acb77a03/extracted/'
# dti = pd.date_range('1958-01', '2001-1', freq='M') # 516
# localList = [path_local+'Snowf_WFD_GPCC_%s.nc.gz' % time for time in dti.strftime('%Y%m')]
# remoteList = [path_watch+'Snowf_WFD_GPCC_%s.nc.gz' % time for time in dti.strftime('%Y%m')]
# DownloadFromURL(remoteList, localList, showLog=True)